In [2]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers

#Import the helper functions
#from helperFunctions import CSP
from helperFunctions import GetCombinedData_HaLT as GetHALT


enableDropout = True;
dropoutPercentage = 0.3;

#Run GetCombinedData to pull the datasets from multiple subjects into a single set
Data, Targets, DataCSP, TargetsCSP = GetHALT('../../../matDown/HaLT_Data', True);

## Split into train and test sets
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(Data, Targets, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))


Processing dataset 1 of 29
Processing dataset 2 of 29
Processing dataset 3 of 29
Processing dataset 4 of 29
Processing dataset 5 of 29
Processing dataset 6 of 29
Processing dataset 7 of 29
Processing dataset 8 of 29
Processing dataset 9 of 29
Processing dataset 10 of 29
Processing dataset 11 of 29
Processing dataset 12 of 29
Processing dataset 13 of 29
Processing dataset 14 of 29
Processing dataset 15 of 29
Processing dataset 16 of 29
Processing dataset 17 of 29
Processing dataset 18 of 29
Processing dataset 19 of 29
Processing dataset 20 of 29
Processing dataset 21 of 29
Processing dataset 22 of 29
Processing dataset 23 of 29
Processing dataset 24 of 29
Processing dataset 25 of 29
Processing dataset 26 of 29
Processing dataset 27 of 29
Processing dataset 28 of 29
Processing dataset 29 of 29


/Users/daniel/Documents/GitHub/ECE228ECGProject/JupyterNotebooks/helperFunctions.py:83: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


In [3]:

## Construct the model
LSTM_EEG = Sequential()
LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(LSTM((50), return_sequences=False))
if(enableDropout):
    LSTM_EEG.add(Dropout(dropoutPercentage))
LSTM_EEG.add(Dense((5),activation='sigmoid'))

LSTM_EEG.summary()
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           120800    
_________________________________________________________________
dropout (Dropout)            (None, 21, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 255       
Total params: 151,255
Trainable params: 151,255
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
967/967 - 18s - loss: 0.4822 - accuracy: 0.3045
Epoch 2/30
967/967 - 17s - loss: 0.4602 - accuracy: 0.349

# Results without CSP

### Training Performance:
Correct MI Prediction: 11152

Incorrect MI Prediction: 4314

Percent Accuracy: 72.107%
#################################


### Testing Performance:
Correct MI Prediction: 4200

Incorrect MI Prediction: 2429

Percent Accuracy: 63.358%
#################################


We can see that training the LSTM on the combined data reduces overfitting but still provides only mediocre test results.

In the next cell, we train the same model on the CSP data, which has already been generated

In [4]:
## Split into train and test sets
DataTrain, DataTest, TargetsTrain, TargetsTest = train_test_split(DataCSP, TargetsCSP, test_size=0.3, random_state=0)

## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

history = LSTM_EEG.fit(DataTrain, TargetsTrain, epochs=30,verbose=2, batch_size=16)

predictionsTest = LSTM_EEG.predict(DataTest)

predictionsTest[predictionsTest>0.5] = 1
predictionsTest[predictionsTest <= 0.5] = 0
comparisonArrayTest = predictionsTest == TargetsTest

correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = DataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(DataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == TargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = DataTrain.shape[0] - correctCountTrain


# In[87]:
trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
print("#################################")
print("#################################")

Epoch 1/30
967/967 - 16s - loss: 0.4712 - accuracy: 0.3538
Epoch 2/30
967/967 - 16s - loss: 0.4099 - accuracy: 0.4938
Epoch 3/30
967/967 - 17s - loss: 0.3693 - accuracy: 0.5727
Epoch 4/30
967/967 - 17s - loss: 0.3311 - accuracy: 0.6243
Epoch 5/30
967/967 - 17s - loss: 0.3148 - accuracy: 0.6443
Epoch 6/30
967/967 - 16s - loss: 0.2932 - accuracy: 0.6761
Epoch 7/30
967/967 - 16s - loss: 0.2864 - accuracy: 0.6812
Epoch 8/30
967/967 - 17s - loss: 0.2750 - accuracy: 0.7003
Epoch 9/30
967/967 - 17s - loss: 0.2613 - accuracy: 0.7196
Epoch 10/30
967/967 - 17s - loss: 0.2488 - accuracy: 0.7339
Epoch 11/30
967/967 - 17s - loss: 0.2364 - accuracy: 0.7454
Epoch 12/30
967/967 - 17s - loss: 0.2327 - accuracy: 0.7543
Epoch 13/30
967/967 - 16s - loss: 0.2214 - accuracy: 0.7670
Epoch 14/30
967/967 - 16s - loss: 0.2153 - accuracy: 0.7791
Epoch 15/30
967/967 - 16s - loss: 0.2060 - accuracy: 0.7873
Epoch 16/30
967/967 - 16s - loss: 0.2039 - accuracy: 0.7882
Epoch 17/30
967/967 - 17s - loss: 0.2034 - accura

# Results with CSP
---
### Training Performance:
Correct MI Prediction: 14178

Incorrect MI Prediction: 1288

Percent Accuracy: 91.672%

---

### Testing Performance:
Testing Performance:
    
Correct MI Prediction: 5934

Incorrect MI Prediction: 695

Percent Accuracy: 89.516%

---
We see that CSP greatly increases the accuracy for both the training and testing sets.

In [5]:
trainAccuracyNoCSP

array([[97.14285714, 94.7107438 , 96.63865546, 94.7107438 , 97.68595041,
        86.12521151, 88.09917355, 85.95890411, 91.86046512, 81.15702479]])

In [6]:
trainAccuracyWithCSP

array([[98.48739496, 91.90082645, 97.14285714, 87.27272727, 92.56198347,
        96.27749577, 98.34710744, 74.14383562, 87.20930233, 81.65289256]])

In [7]:
combinedArray = np.transpose(np.asarray(np.vstack((trainAccuracyNoCSP,testAccuracyNoCSP,trainAccuracyWithCSP, testAccuracyWithCSP))))

In [8]:
combinedArray.shape

(10, 4)

In [11]:
np.savetxt('5FperSubject_Dropout.csv', combinedArray, delimiter=',')

In [10]:
pwd

'/home/davalenc/ProjectFiles/ECE228ECGProject/JupyterNotebooks'

In [3]:
import numpy as np
a = np.zeros((3,2,3))+5

In [5]:
a = a[:, :, 0:2]

In [6]:
a

array([[[5., 5.],
        [5., 5.]],

       [[5., 5.],
        [5., 5.]],

       [[5., 5.],
        [5., 5.]]])